In [2]:
%cd /Midgard/home/amirme/an_explanation_model

/Midgard/home/amirme/an_explanation_model


In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
from functools import partial

from source.project_manager import (
    load_experiment_inconsistency,
    load_experiment_metadata,
    delete_experiment_data,
    check_file_exists,
)
from commands.experiment_6_1 import (
    save_metadata_dir,
    dataset_dir,
    input_shape,
    save_output_dir,
)

os.makedirs(save_output_dir, exist_ok=True)
from source.data_manager import (
    preprocess_masks,
    plot_masks,
    minmax_normalize,
    sum_channels,
    symmetric_minmax_normalize,
    single_query_imagenet,
    save_axis,
)
from source.labels import IMAGENET_LABELS

imshow_args = {"cmap": "jet"}

query_imagenet = partial(
    single_query_imagenet, dataset_dir=dataset_dir, input_shape=input_shape
)

2023-11-07 12:24:14.654238: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 12:24:20.418693: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
project_metadata = load_experiment_metadata(save_metadata_dir)
print(project_metadata.columns)
project_metadata = project_metadata.set_index(
    [
        "stream_name",
        "stream_statistic",
        "image_index",
        "baseline_mask_type",
        "alpha_mask_value",
    ]
).sort_index()
print(project_metadata.shape)

Index(['time_to_compute', 'batch_index', 'monitored_statistic_change',
       'path_prefix', 'data_path', 'stream_name', 'stream_statistic',
       'alpha_mask_type', 'alpha_mask_value', 'baseline_mask_type',
       'projection_type', 'projection_distribution', 'projection_top_k',
       'projection_index', 'label', 'architecture', 'method', 'output_layer',
       'dataset', 'image_index', 'image_path', 'input_shape', 'num_classes',
       'seed', 'batch_size', 'max_batches', 'min_change',
       'metadata_file_path'],
      dtype='object')
(972, 23)


In [ ]:
vc = project_metadata.index.value_counts()
print(vc)
print(len(vc))
image_indices = project_metadata.index.get_level_values("image_index").unique()

stream_name            stream_statistic  image_index  baseline_mask_type  alpha_mask_value
log_probs              meanx             17           gaussian            0.00                1
vanilla_grad_mask      meanx2            17           gaussian            0.20                1
results_at_projection  meanx             70           gaussian            0.73                1
                                                                          0.74                1
                                                                          0.75                1
                                                                                             ..
                                         17           gaussian            0.04                1
                                                                          0.05                1
                                                                          0.06                1
                                             

In [ ]:
for name in project_metadata.index.names:
    print(
        name,
        "=",
        project_metadata.index.get_level_values(name).value_counts().to_dict(),
    )

stream_name = {'log_probs': 324, 'results_at_projection': 324, 'vanilla_grad_mask': 324}
stream_statistic = {'meanx': 648, 'meanx2': 324}
image_index = {17: 243, 43: 243, 60: 243, 70: 243}
baseline_mask_type = {'gaussian': 972}
alpha_mask_value = {0.0: 12, 0.41: 12, 0.59: 12, 0.58: 12, 0.57: 12, 0.56: 12, 0.55: 12, 0.54: 12, 0.53: 12, 0.52: 12, 0.51: 12, 0.5: 12, 0.49: 12, 0.48: 12, 0.47: 12, 0.46: 12, 0.45: 12, 0.44: 12, 0.43: 12, 0.6: 12, 0.61: 12, 0.62: 12, 0.72: 12, 0.79: 12, 0.78: 12, 0.77: 12, 0.76: 12, 0.75: 12, 0.74: 12, 0.73: 12, 0.71: 12, 0.63: 12, 0.7: 12, 0.69: 12, 0.68: 12, 0.67: 12, 0.66: 12, 0.65: 12, 0.64: 12, 0.42: 12, 0.4: 12, 0.01: 12, 0.39: 12, 0.18: 12, 0.17: 12, 0.16: 12, 0.15: 12, 0.14: 12, 0.13: 12, 0.12: 12, 0.11: 12, 0.1: 12, 0.09: 12, 0.08: 12, 0.07: 12, 0.06: 12, 0.05: 12, 0.04: 12, 0.03: 12, 0.02: 12, 0.19: 12, 0.2: 12, 0.21: 12, 0.31: 12, 0.38: 12, 0.37: 12, 0.36: 12, 0.35: 12, 0.34: 12, 0.33: 12, 0.32: 12, 0.3: 12, 0.22: 12, 0.29: 12, 0.28: 12, 0.27: 12, 

In [ ]:
import functools


def spectral_lens_argmax(data):
    grad_max = np.load(data.iloc[0]["grad_mask"])
    grad_max = sum_channels(grad_max)
    grad_argmax = np.ones_like(grad_max) * -1
    for id, row in data.iterrows():
        temp_grad = np.load(row["grad_mask"])
        temp_grad = sum_channels(temp_grad)
        grad_max_id = temp_grad > grad_max
        grad_argmax[grad_max_id] = row["alpha_mask_value"]
        grad_max = np.maximum(grad_max, temp_grad)

    assert (grad_argmax > -1).all()

    assert grad_argmax.ndim == 3, f"{grad_argmax.shape} must be 4d (H,W,C)"
    return grad_argmax


def preprocess_masks(masks, preprocesses):
    for preprocess in preprocesses:
        masks = preprocess(masks)
    return masks


def spectral_lens_generic(data, func, perprocess):
    init_val = np.load(data.iloc[0]["grad_mask"])
    init_val = preprocess_masks(init_val, preprocesses=perprocess)

    for id, row in data.iterrows():
        temp_grad = np.load(row["grad_mask"])
        temp_grad = preprocess_masks(temp_grad, preprocesses=perprocess)
        init_val = func(init_val, temp_grad)

    assert init_val.ndim == 3, f"{init_val.shape} must be 4d (H,W,C)"

    return init_val


def spectral_lens_max(data):
    grad_max = spectral_lens_generic(data, np.maximum, [sum_channels])
    return grad_max


def running_sum(init_val, temp_grad):
    return init_val + temp_grad


def div_by_sum(grad_mask, grad_sum):
    return grad_mask / grad_sum


def spectral_lens_negentropy(data):
    def running_negentropy(init_val, temp_grad):
        return init_val + temp_grad * np.log(temp_grad)

    grad_sum = spectral_lens_generic(data, running_sum, [sum_channels])
    div_by_sum_internal = functools.partial(div_by_sum, grad_sum=grad_sum)
    grad_negentropy = spectral_lens_generic(
        data, running_negentropy, [sum_channels, div_by_sum_internal]
    )

    assert grad_negentropy.ndim == 3, f"{grad_negentropy.shape} must be 4d (H,W,C)"
    grad_negentropy = np.expand_dims(grad_negentropy, axis=0)
    return -grad_negentropy


def spectral_lens_sum(data):
    grad_sum = spectral_lens_generic(data, running_sum, [sum_channels])
    grad_sum = np.expand_dims(grad_sum, axis=0)
    return grad_sum


def spectral_lens_mean_freq(data):
    def func(init_val, temp_grad, frequency):
        return init_val + temp_grad * frequency

    grad_sum = spectral_lens_generic(data, running_sum, [sum_channels])
    div_by_sum_internal = functools.partial(div_by_sum, grad_sum=grad_sum)
    perprocess = [sum_channels, div_by_sum_internal]

    init_val = np.load(data.iloc[0]["grad_mask"])
    init_val = np.zeros_like(sum_channels(init_val))

    for id, row in data.iterrows():
        temp_grad = np.load(row["grad_mask"])
        temp_grad = preprocess_masks(temp_grad, preprocesses=perprocess)
        init_val = func(init_val, temp_grad, row["alpha_mask_value"])

    assert init_val.ndim == 3, f"{init_val.shape} must be 4d (H,W,C)"
    init_val = np.expand_dims(init_val, axis=0)
    return init_val


def spectral_lens_negvar(data):
    grad_mean = spectral_lens_generic(data, running_sum, [sum_channels])
    grad_var = spectral_lens_generic(
        data, running_sum, [sum_channels, lambda x: x**2]
    )
    grad_mean = grad_mean / len(data)
    grad_var = grad_var / len(data) - grad_mean**2
    grad_var = np.expand_dims(grad_var, axis=0)
    return grad_var

In [ ]:
image_indices = project_metadata.index.get_level_values("image_index").unique()
explanations_temp = project_metadata.loc[
    ("vanilla_grad_mask", "meanx2", image_indices, "gaussian", slice(None)), "data_path"
]
explanations_temp = explanations_temp.droplevel(
    ["stream_name", "stream_statistic", "baseline_mask_type"]
)
explanations_temp.name = "grad_mask"
explanations_temp.sort_index(inplace=True)
explanations_temp = explanations_temp.reset_index()
# explanations_argmax = explanations_temp.groupby("image_index", as_index=False).apply(
#     spectral_lens_argmax
# )
# explanations_max = explanations_temp.groupby("image_index",as_index=False).apply(spectral_lens_max)
explanations_negentropy = explanations_temp.groupby(
    "image_index", as_index=False
).apply(spectral_lens_negentropy)
# explanations_mean = explanations_temp.groupby("image_index",as_index=False).apply(spectral_lens_mean)
# explanations_negvar = explanations_temp.groupby("image_index",as_index=False).apply(spectral_lens_negvar)
# explanations_sum = explanations_temp.groupby("image_index", as_index=False).apply(
#     spectral_lens_sum
# )
explanations_mean_freq = explanations_temp.groupby(
    "image_index", as_index=False
).apply(spectral_lens_mean_freq)

In [ ]:
# original_image, label, image_path = query_imagenet(image_index=43)

2023-11-07 11:43:39.899464: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-11-07 11:43:39.899515: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: gondor
2023-11-07 11:43:39.899526: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: gondor
2023-11-07 11:43:39.899610: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 525.89.2
2023-11-07 11:43:39.899632: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 525.89.2
2023-11-07 11:43:39.899638: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 525.89.2
2023-11-07 11:43:40.021939: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (

In [ ]:
n_cols = 3
n_rows = len(image_indices)
fig, axes = plt.subplots(
    n_rows,
    n_cols,
    figsize=(5 * n_cols, 5 * n_rows),
)
axes = axes.flatten()
i = 0
for image_index, explanation_negentropy,explanation_mean_freq in zip(image_indices, explanations_negentropy,explanations_mean_freq):
    if i >= axes.size:
        break
    original_image, label, image_path = query_imagenet(image_index=image_index)

    axes[0].imshow(np.squeeze(original_image))
    axes[0].set_title(f"ImageNet label: {IMAGENET_LABELS[label]}")

    explanation_negentropy = minmax_normalize(explanation_negentropy)
    axes[1].imshow(explanation_negentropy.squeeze(), cmap="jet")
    axes[1].set_title("negative entropy")

    explanation_mean_freq = minmax_normalize(explanation_mean_freq)
    axes[2].imshow(explanation_mean_freq.squeeze(), cmap="jet")
    axes[2].set_title("mean frequency")

    i += 3

plt.show()

NameError: name 'image_indices' is not defined